In [5]:
import pandas

In [ ]:
##Boruta is a pretty smart algorithm dating back to 2010 designed to automatically perform feature selection on a dataset. It was born as a package for R (this is the article that first described it). A version of Boruta for Python — called BorutaPy — exists and can be found here.

In [7]:
import numpy as np### make X_shadow by randomly permuting each column of X
np.random.seed(42)
X_shadow = X.apply(np.random.permutation)
X_shadow.columns = ['shadow_' + feat for feat in X.columns]### make X_boruta by appending X_shadow to X
X_boruta = pd.concat([X, X_shadow], axis = 1)

In [8]:
from sklearn.ensemble import RandomForestRegressor### fit a random forest (suggested max_depth between 3 and 7)
forest = RandomForestRegressor(max_depth = 5, random_state = 42)
forest.fit(X_boruta, y)### store feature importances
feat_imp_X = forest.feature_importances_[:len(X.columns)]
feat_imp_shadow = forest.feature_importances_[len(X.columns):]### compute hits
hits = feat_imp_X > feat_imp_shadow.max()

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
train = pd.read_csv("C:/Users/iDIGITAL/Desktop/Challenge/Broute/test_X.csv")
train.shape

(1005, 45)

In [3]:
train.columns

Index(['0', 'SID', 'NAME', 'ISO', 'YEAR', 'BASIN', 'SUB BASIN', 'NATURE',
       'ISO_TIME', 'COORDS', 'TOTAL_HRS', 'DAY_HRS', 'NIGHT_HRS', 'USA_SSHS',
       'WIND_CALC_MEAN', 'PRES_CALC_MEAN', 'STORM_SPD_MEAN', 'STORM_DR_MEAN',
       'V_LAND_KN', '34KN_POP', '34KN_ASSETS', '64KN_POP', '64KN_ASSETS',
       '96KN_POP', '96KN_ASSETS', 'CPI', 'TOTAL_DAMAGE_(000$)', 'TOTAL_DEATHS',
       'Air transport, freight (million ton-km)',
       'Arable land (hectares per person)', 'Cereal yield (kg per hectare)',
       'Food production index (2004-2006 = 100)', 'GDP growth (annual %)',
       'GDP per capita (constant 2010 US$)', 'Net flows from UN agencies US$',
       'Life expectancy at birth, total (years)',
       'Mobile cellular subscriptions (per 100 people)',
       'Population density (people per sq. km of land area)',
       'Adjusted savings: education expenditure (% of GNI)',
       'Rural population (% of total population)', 'Income_level_Final',
       'pop_max_34_adj', 'pop_ma

In [23]:
#All categorical values will be one-hot encoded.
train = pd.get_dummies(train, drop_first=True, dummy_na=True)
train.shape

(1005, 3446)

In [24]:
#Get all feature names from the dataset
features = [f for f in train.columns if f not in ['TOTAL_AFFECTED']]
len(features)

3445

In [25]:
#Replace all missing values with the Mean.
train[features] = train[features].fillna(train[features].mean()).clip(-1e9,1e9)

In [26]:
#Get the final dataset X and labels Y
X = train[features].values
Y = train['TOTAL_AFFECTED'].values.ravel()

In [27]:
#Next we setup the RandomForrestClassifier as the estimator to use for Boruta. The max_depth of the tree is advised on the Boruta Github page to be between 3 to 7.
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

In [28]:
#Next we setup Boruta. It uses the scikit-learn interface as much as possible so we can use fit(X, y), transform(X), fit_transform(X, y). I'll let it run for a maximum of max_iter = 50 iterations. With perc = 90 a threshold is specified. The lower the threshold the more features will be selected. I usually use a percentage between 80 and 90.
boruta_feature_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=4242, max_iter = 50, perc = 90)
boruta_feature_selector.fit(X, Y)

Iteration: 	1 / 50
Confirmed: 	0
Tentative: 	3445
Rejected: 	0
Iteration: 	2 / 50
Confirmed: 	0
Tentative: 	3445
Rejected: 	0
Iteration: 	3 / 50
Confirmed: 	0
Tentative: 	3445
Rejected: 	0
Iteration: 	4 / 50
Confirmed: 	0
Tentative: 	3445
Rejected: 	0
Iteration: 	5 / 50
Confirmed: 	0
Tentative: 	3445
Rejected: 	0
Iteration: 	6 / 50
Confirmed: 	0
Tentative: 	3445
Rejected: 	0
Iteration: 	7 / 50
Confirmed: 	0
Tentative: 	3445
Rejected: 	0
Iteration: 	8 / 50
Confirmed: 	0
Tentative: 	1683
Rejected: 	1762
Iteration: 	9 / 50
Confirmed: 	0
Tentative: 	1683
Rejected: 	1762
Iteration: 	10 / 50
Confirmed: 	0
Tentative: 	1683
Rejected: 	1762
Iteration: 	11 / 50
Confirmed: 	29
Tentative: 	1654
Rejected: 	1762
Iteration: 	12 / 50
Confirmed: 	29
Tentative: 	1107
Rejected: 	2309
Iteration: 	13 / 50
Confirmed: 	29
Tentative: 	1107
Rejected: 	2309
Iteration: 	14 / 50
Confirmed: 	29
Tentative: 	1107
Rejected: 	2309
Iteration: 	15 / 50
Confirmed: 	29
Tentative: 	1107
Rejected: 	2309
Iteration: 	16 / 50


BorutaPy(alpha=0.05,
         estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                          class_weight='balanced',
                                          criterion='gini', max_depth=5,
                                          max_features='auto',
                                          max_leaf_nodes=None, max_samples=None,
                                          min_impurity_decrease=0.0,
                                          min_impurity_split=None,
                                          min_samples_leaf=1,
                                          min_samples_split=2,
                                          min_weight_fraction_leaf=0.0,
                                          n_estimators=162, n_jobs=-1,
                                          oob_score=False,
                                          random_state=RandomState(MT19937) at 0xC9DBDB0,
                                          verbose=0, warm_start=False

In [29]:
#After Boruta has run we can transform our dataset.
X_filtered = boruta_feature_selector.transform(X)
X_filtered.shape

(1005, 31)

In [30]:
#And we create a list of the feature names if we would like to use them at a later stage.
final_features = list()
indexes = np.where(boruta_feature_selector.support_ == True)
for x in np.nditer(indexes):
    final_features.append(features[x])
print(final_features)

['0', 'YEAR', 'TOTAL_HRS', 'NIGHT_HRS', 'USA_SSHS', 'WIND_CALC_MEAN', 'PRES_CALC_MEAN', 'STORM_SPD_MEAN', 'STORM_DR_MEAN', 'V_LAND_KN', '34KN_POP', '64KN_POP', '96KN_POP', 'CPI', 'TOTAL_DAMAGE_(000$)', 'TOTAL_DEATHS', 'Air transport, freight (million ton-km)', 'Arable land (hectares per person)', 'Cereal yield (kg per hectare)', 'Food production index (2004-2006 = 100)', 'GDP growth (annual %)', 'GDP per capita (constant 2010 US$)', 'Net flows from UN agencies US$', 'Life expectancy at birth, total (years)', 'Mobile cellular subscriptions (per 100 people)', 'Population density (people per sq. km of land area)', 'Adjusted savings: education expenditure (% of GNI)', 'Rural population (% of total population)', 'pop_max_34_adj', 'pop_max_50_adj', 'pop_max_64_adj']
